In [1]:
import numpy as np
from syft.core.tensor.autodp.phi_tensor import PhiTensor
from syft.core.tensor.autodp.gamma_tensor import GammaTensor
from syft.core.tensor.fixed_precision_tensor import FixedPrecisionTensor as FPT
from syft.core.adp.data_subject_list import DataSubjectList
from syft.core.tensor.lazy_repeat_array import lazyrepeatarray as lra

/home/shubham/anaconda3/envs/syft/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Torch Dataset equivalent

We have to implement:
- padding -> DONE!
- random horizontal flip -> DONE!
- random vertical flip -> DONE!
- random rotation -> Done but not confident about resultant LRA, or what to do with 0 values. -> Might try training the model without random rotations to see how testing accuracy suffers.
- normalize -> DONE!

In [2]:
a = PhiTensor(child=np.random.random((5,5)),data_subjects=np.ones((5,5)), min_vals=0, max_vals=1)

In [3]:
a

PhiTensor(child=FixedPrecisionTensor(child=[[34210  1906 58510 62504 56966]
 [12712  7041  1395 63098 22441]
 [64616 58635 29876 46437 10093]
 [46903 47300 52699 53799 50414]
 [25252  7880 34860 28419 63987]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [4]:
a.pad(width=64)

PhiTensor(child=FixedPrecisionTensor(child=[[34210  1906 58510 ... 58510 62504 56966]
 [12712  7041  1395 ...  1395 63098 22441]
 [64616 58635 29876 ... 29876 46437 10093]
 ...
 [64616 58635 29876 ... 29876 46437 10093]
 [46903 47300 52699 ... 52699 53799 50414]
 [25252  7880 34860 ... 34860 28419 63987]]), min_vals=<lazyrepeatarray data: 0.0212860107421875 -> shape: (133, 133)>, max_vals=<lazyrepeatarray data: 0.9859619140625 -> shape: (133, 133)>)

In [5]:
a.random_horizontal_flip()

PhiTensor(child=FixedPrecisionTensor(child=[[56966 62504 58510  1906 34210]
 [22441 63098  1395  7041 12712]
 [10093 46437 29876 58635 64616]
 [50414 53799 52699 47300 46903]
 [63987 28419 34860  7880 25252]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [6]:
a.random_vertical_flip()

PhiTensor(child=FixedPrecisionTensor(child=[[25252  7880 34860 28419 63987]
 [46903 47300 52699 53799 50414]
 [64616 58635 29876 46437 10093]
 [12712  7041  1395 63098 22441]
 [34210  1906 58510 62504 56966]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [7]:
a.random_rotation(degrees=20)

PhiTensor(child=FixedPrecisionTensor(child=[[    0     0     0     0     0     0]
 [    0  2175 32000 62612 31720     0]
 [    0 19085  3074 39272 21785     0]
 [    0 67975 55309 52356 50480     0]
 [    0 42142 31328 41382 33777     0]
 [    0     0     0     0     0     0]]), min_vals=<lazyrepeatarray data: 0.0 -> shape: (6, 6)>, max_vals=<lazyrepeatarray data: 1.037225483235559 -> shape: (6, 6)>)

In [8]:
a.normalize(mean=0.5, std=0.5)

PhiTensor(child=FixedPrecisionTensor(child=[[  2884 -61724  51484  59472  48396]
 [-40112 -51454 -62746  60660 -20654]
 [ 63696  51734  -5784  27338 -45350]
 [ 28270  29064  39862  42062  35292]
 [-15032 -49776   4184  -8698  62438]]), min_vals=<lazyrepeatarray data: -1.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 1.0 -> shape: ()>)

In [9]:
a.child.decode()

array([[0.52200317, 0.02908325, 0.89279175, 0.95373535, 0.86923218],
       [0.19396973, 0.10743713, 0.02128601, 0.96279907, 0.34242249],
       [0.98596191, 0.8946991 , 0.45587158, 0.70857239, 0.15400696],
       [0.71568298, 0.72174072, 0.80412292, 0.82090759, 0.76925659],
       [0.38531494, 0.12023926, 0.53192139, 0.43363953, 0.97636414]])

<hr>
<hr>

## Model Training


To Do:
- Implement the layers needed for a ConvNet for DP Tensors:
    - Conv layer
    - BatchNorm2D
    - LeakyReLU
    - AvgPool2d
    - Linear
- Do 1 forward pass with these layers
- Do 1 backprop with these layers

In [10]:
from syft.core.tensor.nn.conv_layers import Conv2d
import torch

In [11]:
import cv2

# Update image path below
img_path = "/home/shubham/PySyft/notebooks/medical-federated-learning-program/network-operators/archive/10253/0/10253_idx5_x1051_y651_class0.png"
img = cv2.imread(img_path)
img = cv2.resize(img, (50, 50))

In [12]:
dp_tensor = PhiTensor(child=img, data_subjects=[1]*img.shape[0], min_vals=0, max_vals=255)

In [13]:
dp_tensor.max_vals.shape

(50, 50, 3)

In [14]:
from syft.core.tensor.nn.batch_norm import BatchNorm2d
from syft import nn

In [15]:
torch.nn.BatchNorm2d(3)(torch.Tensor(img.reshape(1, *img.shape[::-1]))).shape

torch.Size([1, 3, 50, 50])

In [16]:
dpt2 = dp_tensor.copy()
dpt2 = dp_tensor.reshape((1, *dp_tensor.shape[::-1]))
BatchNorm2d(dpt2, 3).child.decode().shape

(1, 3, 50, 50)

In [17]:
torch.nn.BatchNorm2d(3)(torch.Tensor(img.reshape(1, *img.shape[::-1])))

tensor([[[[ 0.0870, -1.5196,  1.0985,  ...,  1.2473,  0.1762, -1.1923],
          [ 1.0985,  0.5927, -0.8353,  ..., -1.7874, -0.3891, -0.2998],
          [-1.8469, -0.3593,  0.2952,  ...,  0.0572, -1.7279, -0.0023],
          ...,
          [-1.4304,  0.1167,  0.6523,  ...,  0.1762, -1.2816, -0.1808],
          [ 0.5332, -0.8651,  1.1580,  ...,  1.2770,  0.7415, -0.4188],
          [ 1.2473,  0.8308, -0.2105,  ...,  0.5035,  1.3960,  0.4440]],

         [[-0.8683,  0.8349,  0.5916,  ..., -0.0471, -1.5982,  0.1354],
          [ 0.2875, -1.1724,  0.6524,  ...,  1.4128,  0.0746, -1.5982],
          [ 0.9870,  0.4091, -1.0508,  ..., -1.2028,  1.1086,  0.5308],
          ...,
          [ 1.2303, -0.0471, -1.6286,  ..., -0.2904,  1.2303, -0.3208],
          [-1.7199, -0.3208, -0.0775,  ...,  0.1354, -1.3549,  1.0478],
          [ 0.4395, -0.8683,  1.2607,  ...,  0.9566, -0.0775, -1.8415]],

         [[ 0.9140,  0.1708, -1.1032,  ...,  0.3300,  1.0201,  1.1528],
          [ 0.5424,  1.2059,  

In [18]:
BatchNorm2d(dpt2, 3).child.decode()

array([[[[ 0.08695984, -1.51960754,  1.09851074, ...,  1.24726868,
           0.17622375, -1.19233704],
         [ 1.09851074,  0.59274292, -0.83532715, ..., -1.78736877,
          -0.38905334, -0.29978943],
         [-1.84687805, -0.35929871,  0.29522705, ...,  0.0572052 ,
          -1.72787476, -0.00227356],
         ...,
         [-1.43035889,  0.11671448,  0.65223694, ...,  0.17622375,
          -1.28160095, -0.18078613],
         [ 0.53323364, -0.86508179,  1.15802002, ...,  1.27702332,
           0.74150085, -0.41879272],
         [ 1.24726868,  0.83074951, -0.21054077, ...,  0.503479  ,
           1.39602661,  0.44398499]],

        [[-0.86825562,  0.8348999 ,  0.59159851, ..., -0.04708862,
          -1.59819031,  0.13537598],
         [ 0.28744507, -1.1723938 ,  0.65242004, ...,  1.4127655 ,
           0.07455444, -1.59819031],
         [ 0.98696899,  0.40910339, -1.05075073, ..., -1.20281982,
           1.10862732,  0.53076172],
         ...,
         [ 1.23028564, -0.04708862

In [19]:
from syft import nn

In [20]:
nn.AvgPool2d(dp_tensor, 3).child.decode().shape

(50, 16, 1)

In [21]:
from torch.nn import AvgPool2d, MaxPool2d

In [22]:
from torch import Tensor
AvgPool2d(3)(Tensor(dp_tensor.child.decode())).shape

torch.Size([50, 16, 1])

In [23]:
nn.MaxPool2d(dp_tensor, kernel_size=2, stride=2).child.decode()

array([[[183.],
        [191.],
        [194.],
        ...,
        [205.],
        [195.],
        [190.]],

       [[207.],
        [199.],
        [199.],
        ...,
        [193.],
        [200.],
        [196.]],

       [[196.],
        [190.],
        [201.],
        ...,
        [188.],
        [228.],
        [183.]],

       ...,

       [[188.],
        [196.],
        [213.],
        ...,
        [202.],
        [199.],
        [227.]],

       [[195.],
        [195.],
        [192.],
        ...,
        [196.],
        [198.],
        [195.]],

       [[201.],
        [181.],
        [198.],
        ...,
        [188.],
        [169.],
        [230.]]])

In [24]:
MaxPool2d(kernel_size=2, stride=2)(Tensor(dp_tensor.child.decode()))

tensor([[[183.],
         [191.],
         [194.],
         ...,
         [205.],
         [195.],
         [190.]],

        [[207.],
         [199.],
         [199.],
         ...,
         [193.],
         [200.],
         [196.]],

        [[196.],
         [190.],
         [201.],
         ...,
         [188.],
         [228.],
         [183.]],

        ...,

        [[188.],
         [196.],
         [213.],
         ...,
         [202.],
         [199.],
         [227.]],

        [[195.],
         [195.],
         [192.],
         ...,
         [196.],
         [198.],
         [195.]],

        [[201.],
         [181.],
         [198.],
         ...,
         [188.],
         [169.],
         [230.]]])

In [25]:
torch.nn.functional.leaky_relu(Tensor(BatchNorm2d(dpt2, 3).child.decode()))

tensor([[[[ 8.6960e-02, -1.5196e-02,  1.0985e+00,  ...,  1.2473e+00,
            1.7622e-01, -1.1923e-02],
          [ 1.0985e+00,  5.9274e-01, -8.3533e-03,  ..., -1.7874e-02,
           -3.8905e-03, -2.9979e-03],
          [-1.8469e-02, -3.5930e-03,  2.9523e-01,  ...,  5.7205e-02,
           -1.7279e-02, -2.2736e-05],
          ...,
          [-1.4304e-02,  1.1671e-01,  6.5224e-01,  ...,  1.7622e-01,
           -1.2816e-02, -1.8079e-03],
          [ 5.3323e-01, -8.6508e-03,  1.1580e+00,  ...,  1.2770e+00,
            7.4150e-01, -4.1879e-03],
          [ 1.2473e+00,  8.3075e-01, -2.1054e-03,  ...,  5.0348e-01,
            1.3960e+00,  4.4398e-01]],

         [[-8.6826e-03,  8.3490e-01,  5.9160e-01,  ..., -4.7089e-04,
           -1.5982e-02,  1.3538e-01],
          [ 2.8745e-01, -1.1724e-02,  6.5242e-01,  ...,  1.4128e+00,
            7.4554e-02, -1.5982e-02],
          [ 9.8697e-01,  4.0910e-01, -1.0508e-02,  ..., -1.2028e-02,
            1.1086e+00,  5.3076e-01],
          ...,
     

In [26]:
nn.functional.leaky_ReLU(BatchNorm2d(dpt2, 3)).decode()

array([[[[ 8.69598389e-02, -1.51824951e-02,  1.09851074e+00, ...,
           1.24726868e+00,  1.76223755e-01, -1.19171143e-02],
         [ 1.09851074e+00,  5.92742920e-01, -8.34655762e-03, ...,
          -1.78680420e-02, -3.87573242e-03, -2.99072266e-03],
         [-1.84631348e-02, -3.58581543e-03,  2.95227051e-01, ...,
           5.72052002e-02, -1.72729492e-02, -1.52587891e-05],
         ...,
         [-1.42974854e-02,  1.16714478e-01,  6.52236938e-01, ...,
           1.76223755e-01, -1.28021240e-02, -1.80053711e-03],
         [ 5.33233643e-01, -8.63647461e-03,  1.15802002e+00, ...,
           1.27702332e+00,  7.41500854e-01, -4.18090820e-03],
         [ 1.24726868e+00,  8.30749512e-01, -2.09045410e-03, ...,
           5.03479004e-01,  1.39602661e+00,  4.43984985e-01]],

        [[-8.68225098e-03,  8.34899902e-01,  5.91598511e-01, ...,
          -4.57763672e-04, -1.59759521e-02,  1.35375977e-01],
         [ 2.87445068e-01, -1.17187500e-02,  6.52420044e-01, ...,
           1.41276550e

In [27]:
Conv2d(dp_tensor, in_channels=3, out_channels=32, kernel_size=3, padding=2)

PhiTensor(child=FixedPrecisionTensor(child=[[[[  3172700   3006675   5572806 ...   5440363   2306544   1782961]
   [  2041313   2243737   1932857 ...   1118947   -703578   -113941]
   [   669629  -2815743   -953428 ...  -1118387  -1269987   1374710]
   ...
   [ -1190101  -1984215    924543 ...   -627317   -343500    654364]
   [ -2087408  -4668461  -4904955 ...  -7778089  -3751817  -1050513]
   [  -889062  -2286381  -3286564 ...  -1090944   -442061   1697070]]

  [[ -1024293    359870  -6937509 ...  -5566653  -5258428  -5565711]
   [ -1184596  -1462316  -5108196 ...  -6240254  -3199011  -5711045]
   [  3833415   3799981  -4474296 ...  -4440412  -8322822  -7161960]
   ...
   [  3641238   2343433  -2588394 ...  -4750223  -5402390  -7105641]
   [  2982042   5075809   3177069 ...   2963800  -1781727  -2231872]
   [  4841048   3493672    679691 ...   -170373  -4883221  -2314674]]

  [[  3402956   2412450   2399377 ...   2215881  -1757532  -1215475]
   [  1498979   2655988   -483273 ...   10

### Forward Pass

In [28]:
X = dp_tensor.pad(64).random_horizontal_flip().random_vertical_flip().random_rotation(20).normalize(mean=0.5,std=0.5)

In [29]:
x = Conv2d(X, in_channels=3, out_channels=32, kernel_size=3, padding=2)
x = nn.functional.leaky_ReLU(BatchNorm2d(x, 32))
x = nn.MaxPool2d(x, kernel_size=2, stride=2)

In [30]:
x = Conv2d(x, in_channels=32, out_channels=64, kernel_size=3, padding=2)
x = nn.functional.leaky_ReLU(BatchNorm2d(x, 64))
x = nn.MaxPool2d(x, kernel_size=2, stride=2)

In [31]:
x = Conv2d(x, in_channels=64, out_channels=128, kernel_size=3, padding=2)
x = nn.functional.leaky_ReLU(BatchNorm2d(x, 128))
x = nn.MaxPool2d(x, kernel_size=2, stride=2)

In [32]:
x = Conv2d(x, in_channels=128, out_channels=256, kernel_size=3, padding=2)
x = nn.functional.leaky_ReLU(BatchNorm2d(x, 256))
x = nn.MaxPool2d(x, kernel_size=2, stride=2)

In [33]:
x = Conv2d(x, in_channels=256, out_channels=512, kernel_size=3, padding=2)
x = nn.functional.leaky_ReLU(BatchNorm2d(x, 512))
x = nn.MaxPool2d(x, kernel_size=2, stride=2)

In [34]:
x = nn.AvgPool2d(x, 7)

In [35]:
x = nn.Linear(x.reshape((-1, 512 * 1 * 1)), 512 * 1 * 1, 2)

In [36]:
x.child.decode()

array([[-0.29856873,  0.10409546]])